In [12]:
import numpy as np
import matplotlib.pyplot as plt
from pyscf.pbc import gto


数值原子轨道

优点：相比平面波可大幅度减少基函数数量

缺点：但是存在eggbox效应，需要大量密集格点做格点积分

Kohn-Sham 方程:$\hat{H}\psi_i={\epsilon}_i\psi_i$

$\hat{H}=\hat{T}+\hat{V}_{ext}+\hat{V}_{Hatree}+\hat{V}_{xc}+E_{ion}$

给定一组数值原子轨道基组$\{\phi_j\}$,则$\psi_i=\sum_{j}C_{ij}\phi_j$

KS方程转化为一个矩阵方程：$HC=SCE$

问题转化为计算在该基组下的矩阵元










In [2]:
Lenth = 4
Mesh = 10
Image = 2
images= [Image]*3
cell = gto.Cell()
cell.a = np.eye(3)*Lenth
cell.atom = 'He 2 2 2'
cell.mesh = np.array([Mesh]*3)
cell.unit = 'Bohr'
cell.basis = 'unc-sto-3g'
cell.build()




利用pyscf中gto类创建cell对象，构建晶胞

参数 

a:晶胞基矢量

atom:原子种类及坐标

mesh:格点数目

basis:基组形式，此处使用“unc-sto-3g",指3个Gaussian函数拟合一个slater函数

images:考虑镜像数目

最后利用build方法完成晶胞的构建，此处构建了一个正交晶系的He原子晶胞



In [3]:

def get_grids(cell):
    grids_x = np.arange(cell.mesh[0]) / cell.mesh[0]
    grids_y = np.arange(cell.mesh[1]) / cell.mesh[1]
    grids_z = np.arange(cell.mesh[2]) / cell.mesh[2]
    grids=[]
    for x in range(cell.mesh[0]):
        for y in range(cell.mesh[1]):
            for z in range(cell.mesh[2]):
                grid=np.einsum('i,ij->j', (grids_x[x], grids_y[y], grids_z[z]), cell.a)
                grids.append(grid)
    return np.array(grids)

def get_lattice(cell, images):
    Ls=[]
    for x in range(-images[0], images[0]+1):
        for y in range(-images[1], images[1]+1):
            for z in range(-images[2], images[2]+1):
                L = np.einsum('i,ij->j', (x, y, z), cell.a)
                Ls.append(L)    
    return np.array(Ls)

def get_ao(cell,images):
    cell = cell.copy()
    grids = get_grids(cell)
    Ls = get_lattice(cell, images)
    atom = cell.atom_coords()
    ao=0
    for L in Ls:
        cell.atom =[['He', atom[0]+L]]
        cell.build()
        ao += cell.eval_gto('GTOval', grids)
    return ao




$\phi(\mathbf{r})=\frac{1}{\sqrt{N}}\sum_{\mathbf{L}}\mu(\mathbf{r-L})$

$\phi(\mathbf{G})=\int d\mathbf{r}e^{-i\mathbf{Gr}}\phi(\mathbf{r})$

In [5]:
from pyscf.pbc.tools import fft,ifft
from pyscf.pbc import df
def G_vector(cell,mesh):
    x=np.fft.fftfreq(mesh[0],1/mesh[0])
    y=np.fft.fftfreq(mesh[1],1/mesh[1])
    z=np.fft.fftfreq(mesh[2],1/mesh[2])
    b=cell.reciprocal_vectors()
    Gs=[]
    for i in x:
        for j in y:
            for k in z:
                Gs.append(i*b[0]+j*b[1]+k*b[2])
    return np.array(Gs)

def get_aoG_values(cell,images):
    ao = get_ao(cell,images)
    nao = ao.shape[1]
    ao_G = []

    image_sum=(2*images[0]+1)*(2*images[1]+1)*(2*images[2]+1)
    for i in range(nao):
        ao_G.append(cell.vol/8/image_sum*fft(ao[:,i],cell.mesh)) 
    ao_G = np.array(ao_G).T
    return ao_G

def overlap1(cell,images):
    ao = get_ao(cell,images)
    weight = cell.vol/(cell.mesh[0]*cell.mesh[1]*cell.mesh[2])
    overlap = np.einsum('xy,xz->yz',ao, ao,optimize=True)*weight
    return overlap
def overlap2(cell,images):
    cell_1 = cell.copy()
    Ls = get_lattice(cell, images)
    overlap = 0
    atom = cell.atom_coords()
    for L in Ls:
        cell_1.atom =[['He', atom[0]+L]]
        cell_1.build()
        overlap += gto.intor_cross('int1e_ovlp',cell,cell_1)
    image_sum=(2*images[0]+1)/(2*images[1]+1)/(2*images[2]+1)
    return overlap/image_sum
def overlap3(cell,images):
    ao_G = get_aoG_values(cell,images)
    weight = 1/cell.vol
    overlap = np.einsum('xy,xz->yz',ao_G.conj(), ao_G,optimize=True)*weight
    return np.real(overlap)





    

1:$\braket{\phi_i|\phi_j}=\sum_{\mathbf{r}}\omega\phi_i^*(\mathbf{r})\phi_j(\mathbf{r})$

2:$\braket{\phi_i|\phi_j}=\sum_{\mathbf{L}}\int d\mathbf{r}\phi_i^*(\mathbf{r})\phi_j(\mathbf{r-L})$

3:$\braket{\phi_i|\phi_j}=\sum_{\mathbf{G}}\omega\phi_i^*(\mathbf{G})\phi_j(\mathbf{G})$

In [6]:
from pyscf.pbc.dft.numint import eval_ao

def kinetic(cell):
    cell = cell.copy()
    grids = get_grids(cell)
    ao,ao_dx,ao_dy,ao_dz = eval_ao(cell,grids, deriv=1)
    weight = cell.vol/(cell.mesh[0]*cell.mesh[1]*cell.mesh[2])
    t = (np.einsum('xy,xz->yz',ao_dx, ao_dx,optimize=True)
    +np.einsum('xy,xz->yz',ao_dy, ao_dy,optimize=True)
    +np.einsum('xy,xz->yz',ao_dz, ao_dz,optimize=True))*1/2*weight
    return t



$\bra{\phi_i}-\frac{1}{2}\nabla^2\ket{\phi_j}=\frac{1}{2}\sum_{\mathbf{r}}\omega(\nabla\phi_i^*)(\nabla\phi_j)$

In [7]:
def nuc(cell,images):
    atom = cell.atom_coords()
    Gv = G_vector(cell,cell.mesh)
    si = np.exp(-1j*np.einsum('zs,xs->zx', atom, Gv))
    rho_nuc_G = -np.einsum('z,zx->x', cell.atom_charges(), si)

    ao = get_ao(cell,images)
    nao = ao.shape[1]
    pair_ao_real_space = np.einsum('xi,xj->xij', ao.conj(), ao)
    pair_ao_G_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                           cell.mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1/cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    v = w * np.einsum('x,x,xij->ij', rho_nuc_G, coulG, pair_ao_G_space)
    return v

$\bra{\phi_i}V_{ext}\ket{\phi_j}\\
=\sum_{\mathbf{L}}\sum_{\mathbf{L}^{'}}\int d\mathbf{r}\phi_i^*(\mathbf{r})\phi_j(\mathbf{r-L})\frac{Z}{|\mathbf{r-R-L^{'}}|}\\
=\sum_{\mathbf{L}}\sum_{\mathbf{L}^{'}}\int d\mathbf{r}\phi_i^*(\mathbf{r-L^{'}})\phi_j(\mathbf{r-L})\frac{Z}{|\mathbf{r-R}|}\\
=\sum_{\mathbf{G}}\omega\rho(\mathbf{G})V_{ext}(\mathbf{G})$

$\rho(\mathbf{G})=\sum_{\mathbf{L}}\sum_{\mathbf{L}^{'}}\int d\mathbf{r}e^{-i\mathbf{Gr}}\phi_i^*(\mathbf{r-L^{'}})\phi_j(\mathbf{r-L})$

$V_{ext}(\mathbf{G})=\frac{4\pi}{G^2}\int d\mathbf{r}e^{-i\mathbf{Gr}}Z\delta(\mathbf{r-R})$

$\bra{\phi_i\phi_j}r_{12}\ket{\phi_k\phi_l}\\
=\int d\mathbf{r}d\mathbf{r^{'}}\phi_i^*(\mathbf{r})\phi_j(\mathbf{r})r_{12}\phi_k^*(\mathbf{r^{'}})\phi_l(\mathbf{r^{'}})\\
=\sum_{\mathbf{G}}\omega\frac{4\pi}{G^2}\rho_{ij}(\mathbf{G})\rho_{kl}(\mathbf{-G})$

In [8]:
def get_eri(cell):
    Gv = G_vector(cell,cell.mesh)

    ao = get_ao(cell,images)
    nao = ao.shape[1]

    pair_ao_real_space = np.einsum('xi,xj->xij', ao.conj(), ao)
    weight = cell.vol / np.prod(cell.mesh)
    pair_ao_G_space = fft(pair_ao_real_space.reshape(-1,nao**2).T,
                          cell.mesh).T.reshape(-1,nao,nao) * weight
    pair_ao_G_inv_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                               cell.mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    eri = w * np.einsum('x,xij,xkl->ijkl', coulG,pair_ao_G_space,pair_ao_G_inv_space)
    return np.real(eri)
def get_jk(eri,dm):
    j = np.einsum('ijkl,ij->kl',eri,dm)
    k = np.einsum('ijkl,jk->il',eri,dm)
    vhf = j-.5*k
    e = np.einsum('ij,ji',vhf,dm)
    return vhf,e



In [9]:
def get_vxc(cell, dm, xc):
    from pyscf.dft import libxc
    aoR = get_ao(cell,images)
    pair_ao = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    rho = np.einsum('xij,ji->x', pair_ao, dm)
    e_xc, v_xc = libxc.eval_xc(xc, rho,)[:2]
    v_xc = v_xc[0]
    w = cell.vol/(cell.mesh[0]*cell.mesh[1]*cell.mesh[2])
    vxc = w * np.einsum('xij,x->ij', pair_ao, v_xc)
    E_xc = w * np.einsum('x,x', rho, e_xc)
    return vxc, E_xc

In [ ]:
import scipy.linalg
hcore = kinetic(cell) + nuc(cell,images)
s = overlap3(cell,images)
eri = get_eri(cell)
dm = np.zeros_like(s)
vhf, E_coul = get_jk( eri,dm)
fock = hcore + vhf
e, c = scipy.linalg.eigh(fock, s)
dm = 2*c[:,:1].T @ c[:,:1]

E_ewald = cell.ewald()
print('E_ewald =', E_ewald)
E = -E_ewald
dE = 1e99

cycle = 0
while dE > 1e-4:
        cycle += 1
        Elast = E

        fock = hcore + vhf
        e, c = scipy.linalg.eigh(fock, s)
        dm = np.einsum('pi,qi->pq', c[:,:1], c[:,:1].conj()) * 2
        vhf, E_coul = get_jk(eri,dm)

        E_elec = np.einsum('ij,ji', hcore, dm) + E_coul
        E = -E_ewald + E_elec
        dE = abs(E - Elast)
        print('SCF cycle', cycle, 'E(HF) =', E, 'dE =', dE)


In [ ]:
def get_ao_values_ref(cell, kpt):
    grids = get_grids(cell)
    ao = cell.pbc_eval_gto('GTOval', grids, kpt=kpt)
    return np.array(ao)
def get_j(cell, dm, kpts1, kpts2):
    ngrids = np.prod(cell.mesh)
    rho = np.zeros(ngrids, dtype=np.complex)
    for k, kpt in enumerate(kpts1):
        aoR = get_ao_values_ref(cell, kpt)
        pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
        rho += np.einsum('xij,ji->x', pair_ao_real_space, dm[k])
    rho *= 1./len(kpts1)

    rhoG = fft(rho, cell.mesh) * cell.vol / np.prod(cell.mesh)

    w = 1./cell.vol
    nao = cell.nao_nr()
    Gv = cell.get_Gv()
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0
    vj_kpts = []
   
    for k, kpt in enumerate(kpts2):
        aoR = get_ao_values_ref(cell, kpt)
        pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
        pair_ao_G_inv_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,cell.mesh).T.reshape(-1,nao,nao) * cell.vol
        vj = w * np.einsum('x,x,xij->ij', rhoG, coulG, pair_ao_G_inv_space)
        vj_kpts.append(vj)
    vj_kpts = np.array(vj_kpts)
    return vj_kpts
kpts1 = cell.make_kpts([2,1,1])
kpts2 = cell.make_kpts([1,3,1])
dm = cell.pbc_intor('int1e_ovlp', kpts=kpts1)
from pyscf.pbc import scf
a=scf.KRHF(cell, kpts1).get_j(cell, dm, kpts=kpts1, kpts_band=kpts2)
b = get_j(cell, dm, kpts1, kpts2)
print(a-b)